<a href="https://colab.research.google.com/github/honginhwa/pytorch_study/blob/master/efficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
#model = EfficientNet.from_name('efficientnet-b0')


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=83edc14e693d26103ad63c4d66b95ac3115fc539f10154df060dee0b29222bb4
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import json
from PIL import Image


import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.optim import lr_scheduler


import torchvision
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms
#from torchvision import transforms

import cv2
import time
import os
import copy
import random as rand
from random import *
from efficientnet_pytorch import EfficientNet
from torchvision import transforms, datasets

model_name = 'efficientnet-b0'  # b5

image_size = EfficientNet.get_image_size(model_name)
print(image_size)
model = EfficientNet.from_name(model_name, num_classes=7)

224


In [3]:
train_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
])

In [4]:
data_dir = "/content/drive/My Drive/Colab Notebooks/Labeld_Crop_Data"
trDsets = {x: dset.ImageFolder(os.path.join(data_dir, x), train_transforms[x]) for x in ['train', 'val']}
trLoaders = {x: torch.utils.data.DataLoader(trDsets[x], batch_size=5, shuffle=True, num_workers=4) for x in ['train', 'val']}  

teDsets = dset.ImageFolder(os.path.join(data_dir,'test'), transform=test_transforms)
teLoaders = torch.utils.data.DataLoader(teDsets, batch_size=5, shuffle=False, num_workers=4)

In [5]:
print(trDsets['train'].classes)
print(teDsets.classes)

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']


## data split

In [6]:
trDsets_sizes = {x: len(trDsets[x]) for x in ['train', 'val']}
print(trDsets_sizes)
class_names = trDsets['train'].classes
print(class_names)

{'train': 9936, 'val': 2503}
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']


## data loader 선언

In [7]:
print(torch.cuda.is_available())     # GPU 사용 가능 여부
print(torch.cuda.current_device())   # GPU 디바이스의 위치
print(torch.cuda.device_count())     # 사용가능한 GPU 개수
print(torch.cuda.get_device_name(0)) # GPU의 이름
print(torch.cuda.device(0))

True
0
1
Tesla T4


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [9]:
model = EfficientNet.from_name('efficientnet-b0')
num_ftrs = model._fc.in_features
print(num_ftrs)
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)
print(model)

1280
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_

In [10]:

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model.parameters(), 
                         lr = 0.05,
                         momentum=0.9,
                         weight_decay=1e-4)

lmbda = lambda epoch: 0.98739
exp_lr_scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer_ft, lr_lambda=lmbda)

'''
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, patience=5)
'''

'\ncriterion = nn.CrossEntropyLoss()\noptimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)\nexp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, factor=0.1, patience=5)\n'

## 데이타 체크

In [11]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=8):
    
    global_info = []
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict()) # state_dict() = 모델 불러오기 
    best_acc = 0.0
    early_stopping = EarlyStopping(patience=11, verbose=True)  # EarlyStopping = 11번에 큰 변화가 없다면 학습을 그만한다.
    for epoch in range(num_epochs):
        local_info = []
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)                                            #train 시작

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()                                   # list에 train이면 학습 val이면 평가 
            else:
                model.eval()                              
                
 #               if epoch > 0:
 #                   scheduler.step(val_loss)
                    
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in trLoaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()                              # 가중치 변화도 = 0 >> forward 

                # forward
                # tensor는 인풋값이 디폴트는 그래디언트가 안들어가있는데 이 값을  
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)                              # torch.max(output,1)은 아웃풋이 [0.1, 0.2, 0.9]일때 (0.9[값] ,2[인덱스])가 나온다 
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()                              # backward로 최신화로 스텝으로 가자 

                running_loss += loss.item() * inputs.size(0)          # item() = tensor값을 scalar값으로 표기 * input.size(0)      
            
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / trDsets_sizes[phase]
            if phase == 'val':
                val_loss = running_loss / trDsets_sizes['val']
            epoch_acc = running_corrects.double() / trDsets_sizes[phase]

            if phase == 'train':
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)
            else:
                local_info.append(epoch_loss)
                ea = epoch_acc.cpu().numpy()
                local_info.append(ea)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
 
        lr_get = get_lr(optimizer)
        print("Current learning rate : {:.8f}".format(lr_get))
        global_info.append(local_info)
        
        if phase =='val':
            early_stopping(epoch_loss, model)

            if early_stopping.early_stop:
                print("Early stopping")
                break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [13]:

torch.cuda.empty_cache()
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)


Epoch 0/99
----------
train Loss: 2.3311 Acc: 0.2503
val Loss: 1.6823 Acc: 0.4515
Current learning rate : 0.05000000
Validation loss decreased (inf --> 1.682295).  Saving model ...
Epoch 1/99
----------
train Loss: 1.5340 Acc: 0.5072
val Loss: 0.7213 Acc: 0.7691
Current learning rate : 0.05000000
Validation loss decreased (1.682295 --> 0.721258).  Saving model ...
Epoch 2/99
----------
train Loss: 1.0066 Acc: 0.6868
val Loss: 0.8068 Acc: 0.7835
Current learning rate : 0.05000000
EarlyStopping counter: 1 out of 11
Epoch 3/99
----------
train Loss: 0.7387 Acc: 0.7664
val Loss: 0.4451 Acc: 0.8618
Current learning rate : 0.05000000
Validation loss decreased (0.721258 --> 0.445124).  Saving model ...
Epoch 4/99
----------
train Loss: 0.5904 Acc: 0.8162
val Loss: 0.6105 Acc: 0.8102
Current learning rate : 0.05000000
EarlyStopping counter: 1 out of 11
Epoch 5/99
----------
train Loss: 0.5150 Acc: 0.8407
val Loss: 0.4215 Acc: 0.8757
Current learning rate : 0.05000000
Validation loss decreased 

In [19]:
torch.save(model_ft, '/content/drive/My Drive/Colab Notebooks/pyTorch/ef_model_ft.pt')
torch.save(model, '/content/drive/My Drive/Colab Notebooks/pyTorch/ef_model.pt')
torch.save(model_ft.state_dict(), '/content/drive/My Drive/Colab Notebooks/pyTorch/ef_model_ft_state_dict.pt')
torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/pyTorch/ef_model_state_dict.pt')
